In [5]:
from pyspark.sql import SparkSession


In [6]:
from pyspark.ml.classification import LogisticRegression


In [7]:
spark = SparkSession.builder.appName('LR').getOrCreate()


In [8]:
df = spark.read.csv('/content/drive/MyDrive/Spark with Python/Python-and-Spark-for-Big-Data-master/Spark_for_Machine_Learning/Logistic_Regression/titanic.csv', inferSchema=True, header=True)

In [9]:
df.printSchema()

root
 |-- PassengerId: integer (nullable = true)
 |-- Survived: integer (nullable = true)
 |-- Pclass: integer (nullable = true)
 |-- Name: string (nullable = true)
 |-- Sex: string (nullable = true)
 |-- Age: double (nullable = true)
 |-- SibSp: integer (nullable = true)
 |-- Parch: integer (nullable = true)
 |-- Ticket: string (nullable = true)
 |-- Fare: double (nullable = true)
 |-- Cabin: string (nullable = true)
 |-- Embarked: string (nullable = true)



In [10]:
df.columns

['PassengerId',
 'Survived',
 'Pclass',
 'Name',
 'Sex',
 'Age',
 'SibSp',
 'Parch',
 'Ticket',
 'Fare',
 'Cabin',
 'Embarked']

In [14]:
my_cols = df.select(['Survived', 'Pclass','Age','SibSp','Parch', 'Fare',  'Embarked', 'Sex'  ])

In [15]:
my_final_data = my_cols.na.drop()

In [26]:
from pyspark.ml.feature import VectorAssembler, VectorIndexer, StringIndexer, OneHotEncoder

In [36]:
gender_indexer = StringIndexer(inputCol='Sex', outputCol='SexIndex')

gender_encoder = OneHotEncoder(inputCol='SexIndex', outputCol='SexVec')

In [37]:
embark_indexer = StringIndexer(inputCol='Embarked', outputCol='EmbarkIndex')

embark_encoder = OneHotEncoder(inputCol='EmbarkIndex', outputCol='EmbarkVec')

In [38]:
assembler = VectorAssembler(inputCols=['Survived', 'Pclass','Age','SibSp','Parch', 'Fare', 'SexVec', 'EmbarkVec'], outputCol='features')

In [39]:
from pyspark.ml import Pipeline

In [40]:
log_reg_titanic = LogisticRegression(featuresCol='features', labelCol='Survived')

In [41]:
pipeline = Pipeline(stages=[gender_indexer, embark_indexer, gender_encoder, embark_encoder, assembler, log_reg_titanic])

In [42]:
train_data, test_data = my_final_data.randomSplit([0.7,0.3])

In [43]:
fit_model = pipeline.fit(train_data)

In [44]:
results = fit_model.transform(test_data)

In [45]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator

In [46]:
my_eval = BinaryClassificationEvaluator(rawPredictionCol='prediction', labelCol='Survived')

In [49]:
results.select('Survived', 'prediction').show()

+--------+----------+
|Survived|prediction|
+--------+----------+
|       0|       0.0|
|       0|       0.0|
|       0|       0.0|
|       0|       0.0|
|       0|       0.0|
|       0|       0.0|
|       0|       0.0|
|       0|       0.0|
|       0|       0.0|
|       0|       0.0|
|       0|       0.0|
|       0|       0.0|
|       0|       0.0|
|       0|       0.0|
|       0|       0.0|
|       0|       0.0|
|       0|       0.0|
|       0|       0.0|
|       0|       0.0|
|       0|       0.0|
+--------+----------+
only showing top 20 rows



In [50]:
AUC = my_eval.evaluate(results)

In [51]:
AUC

1.0